In [ ]:
#-*- coding: utf-8 -*-
# 라이브러리 import

import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

import datetime
from dateutil.relativedelta import relativedelta
try:
    from urllib import parse
except:
    !pip install urllib3
    from urllib import parse

import time

try:
    import csv
except:
    !pip install csv
    import csv

try:
    import getpass
except:
    !pip install getpass4
    import getpass

import re


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [ ]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )

In [ ]:
# 비밀번호 마스킹 처리 함수
def printPass(password) :
    masking_password = ""
    for s in password :
        masking_password += "·"
    return masking_password

In [ ]:
# 로그인 처리 함수 loginType F = 페이스북 계정 로그인, I = 인스타 계정 로그인
def instaLogin(chrome, loginType="F", email = None, password = None) :
    chrome.get("https://www.instagram.com/accounts/login/")
    time.sleep(3)
    print(chrome.page_source)
    insta_formObj = chrome.find_element_by_css_selector("form[id=loginForm]")
    emailObj = None
    passObj = None
    loginBtnObj = None
    if loginType == "I":
        print("loginType = I")
        emailObj = insta_formObj.find_element_by_css_selector("input[name=username]")
        passObj = insta_formObj.find_element_by_css_selector("input[name=password]")
        loginBtnObj = insta_formObj.find_element_by_css_selector("button[type=submit]")
    else :
        print("loginType = F")
        loginMoveBtnObj = insta_formObj.find_element_by_css_selector("button.sqdOP.yWX7d.y3zKF[type=button]")
        # 오류로 인해 스크립트로 처리
        #loginMoveBtnObj.click()
        chrome.execute_script("arguments[0].click();", loginMoveBtnObj)
        time.sleep(3)
        facebook_formObj = chrome.find_element_by_css_selector("form[id=login_form]")
        emailObj = facebook_formObj.find_element_by_css_selector("input[id=email]")
        passObj = facebook_formObj.find_element_by_css_selector("input[id=pass]")
        loginBtnObj = facebook_formObj.find_element_by_css_selector("button[id=loginbutton]")

    if email is None :
        email = input("이메일 또는 아이디 : ")
    else :
        print("이메일 또는 아이디 : " + email)

    if password is None :
        password = getpass.getpass("비밀번호 : ")
    else :
        print("비밀번호 : " + printPass(password))

    emailObj.clear()
    emailObj.send_keys(email)
    passObj.clear()
    passObj.send_keys(password)
    time.sleep(1)
    # 오류로 인해 스크립트로 처리
    #loginBtnObj.click()
    chrome.execute_script("arguments[0].click();", loginBtnObj)
    time.sleep(5)


In [ ]:
# 페이스북 로그인 처리 체크 후 재처리를 위한 함수
def loginFacebookCheck(chrome) :
    if chrome.current_url == "https://www.instagram.com/accounts/login/":
        insta_formObj = chrome.find_element_by_css_selector("form[id=loginForm]")
        loginMoveBtnObj = insta_formObj.find_element_by_css_selector("button.sqdOP.yWX7d.y3zKF[type=button]")
        # 오류로 인해 스크립트로 처리
        #loginMoveBtnObj.click()
        chrome.execute_script("arguments[0].click();", loginMoveBtnObj)
        time.sleep(5)
        return True
    else :
        return False


In [ ]:
# 첫번째 사진 선택 함수
def selectFirst(chrome):
    try:
        first = chrome.find_element_by_css_selector("div._9AhH0")
        # 오류로 인해 스크립트로 처리
        #first.click()
        chrome.execute_script("arguments[0].click();", first)
        time.sleep(2)
        return True
    except Exception as e:
        print(e)
        return False


In [ ]:
# 다음 사진으로 이동 함수
def moveNext(chrome):
    try:
        right = chrome.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
        # 오류로 인해 스크립트로 처리
        #right.click()
        chrome.execute_script("arguments[0].click();", right)
        time.sleep(3)
        return True
    except Exception as e:
        print(e)
        return False


In [ ]:
# 내용 추출 함수
def getContent(chrome):
    
    html = chrome.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > a > span')[0].text   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    return data

In [ ]:
# 크롤링 시작 함수
def startSearch(loginType = "F", email = None, password = None) :
    search_txt = input("search : ")
    encode_type = "utf-8"
    search_encode = parse.quote(search_txt)
    if __name__ == '__main__' or 'google.colab' in sys.modules:
        chrome = webdriver.Chrome('chromedriver', options=options)
        try:
            instaLogin(chrome, loginType, email, password)
            time.sleep(2)
            
            #print(chrome.current_url)
            searchUrl = "https://www.instagram.com/explore/tags/" + search_encode + "/"
            
            chrome.get(searchUrl)
            time.sleep(3)
            
            # Facebook 로드인 연동이 안되는 경우가 있어 Facebook 재 연동처리
            if loginType == "F" :
                if loginFacebookCheck(chrome) :
                    print("loginFacebookCheck")
                    chrome.get(searchUrl)
                    time.sleep(3)
                    
            
            print(chrome.current_url)
            now_date = datetime.datetime.now()

            cvsFileName = "inta_search_" + search_txt + "_" + now_date.strftime("%Y-%m-%d") + ".csv"
            with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
                colnames = ['content', 'date', 'like', 'place', 'tags']
                w = csv.writer(f)
                w.writerow(colnames)
            count = 0
            if selectFirst(chrome) :
                while True :
                    count = count+1
                    with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                        w = csv.writer(f)
                        data = getContent(chrome)
                        print("[" + str(count) + "] : ", data)
                        w.writerow(data)
                    if moveNext(chrome) :
                        time.sleep(2)
                    else :
                        break;
        except Exception as e:
            print(e)
        finally:
            if chrome:
                chrome.close()

In [ ]:
startSearch()